In [21]:
# imports
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import math
import keras
from scipy.stats  import norm
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras        import backend as K
from datetime     import datetime
from keras.constraints import maxnorm
from keras.optimizers      import RMSprop
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics       import mean_squared_error
from matplotlib.pylab      import rcParams
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
%matplotlib inline
rcParams['figure.figsize'] = 15, 6

# number of inputs to be fed
look_back = 288
# number of outputs to be generated
look_forward = 24

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1, look_forward=2):
    dataX, dataY = [], []
    np.array(dataY)
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back),0]
        if len(dataset[i + look_back:i+look_back+look_forward, 0]) == look_forward:
            dataX.append(a)
            dataY.append(dataset[i + look_back:i+look_back+look_forward, 0])
    return np.array(dataX), np.array(dataY)

def predict_with_uncertainty(model, dataset, n_iter=100):
    # This function activates drop-out when doing predictions
    f = K.function([model.layers[0].input, K.learning_phase()],
                      [model.layers[-1].output])
    all_pred = np.zeros((n_iter,) + (1,look_forward,))
    for iter in range(n_iter):
        all_pred[iter] = f([dataset, 1])
    avg_pred = all_pred.mean(axis=0)
    std = np.std(all_pred, axis=0)
    return all_pred, avg_pred, std

# given a model and data from the original dataset, it uses predict_with_uncertainty to 
def run_predictions (model, dataset):
    # normalizes the dataset and prepares it to be fed to the model
    dataset = np.array(dataset).transpose().reshape(-1, 1)
    dataset = scaler.fit_transform(dataset).transpose()
    data = (dataset.T)
    data = data.reshape(1, 1, data.shape[0])
    # feeds the model and data into a function that will activate the dro-out
    all_pred, avg_pred, std = predict_with_uncertainty(model, data)
    
    # All the predictions
    all_pred = scaler.inverse_transform(all_pred.reshape(all_pred.shape[0], all_pred.shape[-1]))
    # The Average Prediction
    avg_pred = scaler.inverse_transform(avg_pred)
    # The Standard Deviation At Each T Of Prediction
    std = scaler.inverse_transform(std)
    
    return all_pred, avg_pred, std

In [22]:
# The interval between each dataset (original data in 5 minute intervals)
time_grouping = '5min'

# load the dataset
dataframe = pd.read_csv('ignored_assets/paxout_table.csv', engine='python', nrows=288*21)
dataframe['time_bucket'] = pd.to_datetime(dataframe['time_bucket'])
dataframe = dataframe.set_index('time_bucket')
dataframe['total'] = dataframe.sum(axis=1)
dataframe['day_hour'] = dataframe.index.round(time_grouping)

# The data set with the sum of all the cars out at the 
dataframe = dataframe[['total','day_hour']].groupby('day_hour').sum()

dataset_orig = dataframe.values
dataset_orig = dataset_orig.astype('float32')

# scale the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset_orig)

# split into train and test sets
train_size = 288*10
test_size = len(dataset) - train_size
test, train = dataset[0:test_size,:], dataset[test_size:len(dataset)-look_forward,:]

# reshape into X=[t, t-1, t-2,..., t-look_back] and Y=[t+1, t+2,... t+look_forward]
trainX, trainY = create_dataset(train, look_back, look_forward)
testX, testY = create_dataset(test, look_back, look_forward)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [23]:


def top_heavy_loss (y_true, y_pred):
    w = np.arange(1,0, -1./24) ** 2
    w = (w / w.sum())[:,None]
    W = K.variable(value = w)
    return K.dot( K.abs(y_pred-y_true), W)

seed = 11
np.random.seed(seed)

# creates a model but does not compile the model
def create_model():
    rate = 0.2
    model = Sequential()
    model.add(Dropout(rate, input_shape=(1, look_back)))
    lstm = LSTM(256, recurrent_dropout=0.3)
    model.add(lstm)
    model.add(Dropout(rate))
    model.add(Dense(128))
    model.add(Dropout(rate))
    model.add(Dense(128))
    model.add(Dropout(rate))
    model.add(Dense(128))
    model.add(Dense(look_forward))
    model.compile(loss=top_heavy_loss, optimizer=RMSprop(lr=0.0002, rho=0.9, epsilon=1e-08, decay=0.0002), metrics=['accuracy'])
    return model

In [ ]:
# creates the model
model = KerasClassifier(build_fn=create_model, verbose=3)
# Defines the parameters to be grid searched
batch_size = [1, 40, 60]
epochs = [10, 20, 50, 60]

param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(trainX, trainY)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/50
Epoch 1/10
Epoch 1/60
Epoch 1/50
Epoch 1/50
Epoch 1/60
Epoch 1/10
Epoch 1/10
Epoch 1/60
Epoch 1/20
Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 2/10
Epoch 2/10
Epoch 2/20
Epoch 2/10
Epoch 3/10
Epoch 3/20
Epoch 3/10
Epoch 3/10
Epoch 4/10
Epoch 4/20
Epoch 4/10
Epoch 4/10
Epoch 5/20
Epoch 5/10
Epoch 5/10
Epoch 5/10
Epoch 6/10
Epoch 6/20
Epoch 6/10
Epoch 6/10
Epoch 7/10
Epoch 7/20
Epoch 7/10
Epoch 7/10
Epoch 8/10
Epoch 8/20
Epoch 8/10
Epoch 8/10
Epoch 9/10
Epoch 9/20
Epoch 9/10
Epoch 9/10
Epoch 10/10
Epoch 10/20
Epoch 10/10
Epoch 10/10
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/20
Epoch 1/20
Epoch 2/20
Epoch 2/20
Epoch 3/20
Epoch 3/20
Epoch 4/20
Epoch 4/20
Epoch 5/20
Epoch 5/20
Epoch 6/20
Epoch 6/20
Epoch 1/50
Epoch 7/20
Epoch 7/20
Epoch 2/10
Epoch 8/20
Epoch 2/50
Epoch 8/20
Epoch 2/10
Epoch 2/50
Epoch 9/20
Epoch 2/20
Epoch 2/50
Epoch 9/20
Epoch 2/10
Epoch 2/20
Epoch 2

Epoch 27/50
Epoch 14/60
Epoch 49/50
Epoch 28/50
Epoch 24/50
Epoch 8/60
Epoch 15/60
Epoch 50/50
Epoch 29/50
Epoch 25/50
Epoch 16/60
Epoch 30/50
Epoch 8/50
Epoch 26/50
Epoch 17/60
Epoch 8/20
Epoch 31/50
Epoch 27/50
Epoch 18/60
Epoch 8/10
Epoch 32/50
Epoch 28/50
Epoch 19/60
Epoch 33/50
Epoch 29/50
Epoch 20/60
Epoch 34/50
Epoch 30/50
Epoch 21/60
Epoch 35/50
Epoch 8/20
Epoch 8/20
Epoch 22/60
Epoch 31/50
Epoch 8/10
Epoch 36/50
Epoch 23/60
Epoch 32/50
Epoch 8/60
Epoch 37/50
Epoch 24/60
Epoch 33/50
Epoch 38/50
Epoch 25/60
Epoch 34/50
Epoch 39/50
Epoch 26/60
Epoch 40/50
Epoch 35/50
Epoch 27/60
Epoch 41/50
Epoch 36/50
Epoch 28/60
Epoch 42/50
Epoch 29/60
Epoch 37/50
Epoch 8/60
Epoch 43/50
Epoch 38/50
Epoch 30/60
Epoch 44/50
Epoch 39/50
Epoch 31/60
Epoch 40/50
Epoch 45/50
Epoch 32/60
Epoch 41/50
Epoch 46/50
Epoch 33/60
Epoch 1/60
Epoch 42/50
Epoch 47/50
Epoch 34/60
Epoch 48/50
Epoch 43/50
Epoch 35/60
Epoch 49/50
Epoch 44/50
Epoch 36/60
Epoch 50/50
Epoch 45/50
Epoch 37/60
Epoch 2/60
Epoch 46/50
Epo

In [ ]:

# creates a model but does not compile the model
def create_model(dropout_rate=0.0, weight_constraint=0):
    init_mode = 'normal'
    rate = 0.2
    model = Sequential()
    model.add(Dropout(rate, input_shape=(1, look_back)))
    lstm = LSTM(256, kernel_initializer=init_mode, recurrent_dropout=0.3)
    model.add(lstm)
    model.add(Dropout(rate))
    model.add(Dense(128, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint) ))
    model.add(Dropout(rate))
    model.add(Dense(128, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint) ))
    model.add(Dropout(rate))
    model.add(Dense(128, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint) ))
    model.add(Dense(look_forward))
    model.compile(loss=top_heavy_loss, optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0), metrics=['accuracy'])
    return model

# creates the model
model = KerasClassifier(build_fn=create_model, epochs=60, batch_size=60, verbose=3)

# Defines the parameters to be grid searched
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7]

param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(trainX, trainY)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [14]:

# creates a model but does not compile the model
def create_model(init_mode='uniform'):
    rate = 0.2
    model = Sequential()
    model.add(Dropout(rate, input_shape=(1, look_back)))
    lstm = LSTM(256, kernel_initializer=init_mode, recurrent_dropout=0.3)
    model.add(lstm)
    model.add(Dropout(rate))
    model.add(Dense(128, kernel_initializer=init_mode))
    model.add(Dropout(rate))
    model.add(Dense(128, kernel_initializer=init_mode))
    model.add(Dropout(rate))
    model.add(Dense(128, kernel_initializer=init_mode))
    model.add(Dense(look_forward))
    model.compile(loss=top_heavy_loss, optimizer=RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0), metrics=['accuracy'])
    return model

# creates the model
model = KerasClassifier(build_fn=create_model, epochs=60, batch_size=60, verbose=3)

# Defines the parameters to be grid searched
init_mode = ['lecun_uniform', 'normal', 'glorot_normal', 'glorot_uniform', 'he_uniform']

param_grid = dict(init_mode=init_mode)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(trainX, trainY)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/60
Epoch 2/60
Epoch 3/60
Epoch 4/60
Epoch 5/60
Epoch 6/60
Epoch 7/60
Epoch 8/60
Epoch 9/60
Epoch 10/60
Epoch 11/60
Epoch 12/60
Epoch 13/60
Epoch 14/60
Epoch 15/60
Epoch 16/60


KeyboardInterrupt: 

In [ ]:
'''
Best: 0.390570 using {'init_mode': 'glorot_normal'}
0.302554 (0.029239) with: {'init_mode': 'lecun_uniform'}
0.326523 (0.061835) with: {'init_mode': 'normal'}
0.390570 (0.093092) with: {'init_mode': 'glorot_normal'}
0.307269 (0.051499) with: {'init_mode': 'glorot_uniform'}
0.335560 (0.055283) with: {'init_mode': 'he_uniform'}

Best: 0.503733 using {'init_mode': 'normal'}
0.416896 (0.060112) with: {'init_mode': 'lecun_uniform'}
0.503733 (0.059045) with: {'init_mode': 'normal'}
0.420825 (0.020303) with: {'init_mode': 'glorot_normal'}
0.415717 (0.141750) with: {'init_mode': 'glorot_uniform'}
0.427898 (0.149020) with: {'init_mode': 'he_uniform'}

Best: 0.441650 using {'init_mode': 'glorot_uniform'}
0.388605 (0.013350) with: {'init_mode': 'lecun_uniform'}
0.329273 (0.055460) with: {'init_mode': 'normal'}
0.317092 (0.021865) with: {'init_mode': 'glorot_normal'}
0.441650 (0.044662) with: {'init_mode': 'glorot_uniform'}
0.374853 (0.084774) with: {'init_mode': 'he_uniform'}

Best: 0.458939 using {'init_mode': 'normal'}
0.361493 (0.062972) with: {'init_mode': 'lecun_uniform'}
0.458939 (0.147261) with: {'init_mode': 'normal'}
0.401179 (0.086889) with: {'init_mode': 'glorot_normal'}
0.396071 (0.083101) with: {'init_mode': 'glorot_uniform'}
0.372495 (0.114335) with: {'init_mode': 'he_uniform'}
'''


In [29]:

# creates a model but does not compile the model
def create_model(init_mode='uniform', learning_rate=0.0001, dropout_rate=0.0, recurrent_dropout=0.0, weight_constraint=1, decay=0):
    model = Sequential()
    model.add(Dropout(dropout_rate, input_shape=(1, look_back)))
    lstm = LSTM(256, kernel_initializer=init_mode, recurrent_dropout=recurrent_dropout)
    model.add(lstm)
    model.add(Dropout(dropout_rate))
    model.add(Dense(128, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint) ))
    model.add(Dropout(dropout_rate))
    model.add(Dense(128, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint) ))
    model.add(Dropout(dropout_rate))    
    model.add(Dense(128, kernel_initializer=init_mode, kernel_constraint=maxnorm(weight_constraint) ))
    model.add(Dense(look_forward))
    model.compile(loss=top_heavy_loss, optimizer=RMSprop(lr=learning_rate, rho=0.9, epsilon=1e-08, decay=decay), metrics=['accuracy'])
    return model

# creates the model
model = KerasClassifier(build_fn=create_model, epochs=60, batch_size=60, verbose=3)

# Defines the parameters to be grid searched
batch_size = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
epochs = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
decay = [0.0, 0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005]
learning_rate = [0.001, 0.0005, 0.0003, 0.0002, 0.00015, 0.0001, 0.00005]
init_mode = ['uniform', 'lecun_uniform', 'normal', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
weight_constraint = [1, 2, 3, 4, 5]
recurrent_dropout  = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7]
dropout_rate  = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7]

param_grid = dict(batch_size=batch_size, epochs=epochs, decay=decay, learning_rate=learning_rate,init_mode=init_mode, weight_constraint=weight_constraint,recurrent_dropout=recurrent_dropout,dropout_rate=dropout_rate)
def fun():
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
    grid_result = grid.fit(trainX, trainY)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/1


KeyboardInterrupt: 

In [27]:
fun()

NameError: global name 'rate' is not defined